In [7]:
def createIndex(shapefile):
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(2263))
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

In [8]:
def findZone(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if zones.geometry[idx].contains(p):
            return idx
    return None

In [9]:
def processTrips(pid, records):
    import csv
    import pyproj
    import shapely.geometry as geom
    import geopandas as gpd
    import fiona
    neighborhoods = gpd.read_file('neighborhoods.geojson').to_crs(fiona.crs.from_epsg(2263))
    boroughs = gpd.read_file('boroughs.geojson').to_crs(fiona.crs.from_epsg(2263))
    index_neighbourhood,zone_neighbourhood = createIndex('neighborhoods.geojson')
    index_boroughs,zone_boroughs = createIndex('boroughs.geojson')
    if pid ==0:
        next(records)
    for i in records:
        ls = i.split(',')
        lst=[]
        isAllDigits = True
        try:
            l = [ls[5],ls[6],ls[9],ls[10]]#there is a blank line after header,this try catch handles it.
            lst = list(map(lambda x: float(x),l))#there is a blank line after header,this try catch handles it.
        except IndexError:
            pass
        except ValueError:
            isAllDigits = False
        if(lst and isAllDigits ):
            proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
            start = geom.Point(proj(lst[0],lst[1]))
            end = geom.Point(proj(lst[2],lst[3]))
            start_index = findZone(start,index_neighbourhood,zone_neighbourhood)
            end_index = findZone(end,index_boroughs,zone_boroughs)
            if None not in [start_index, end_index] :
                #print(count,boroughs['boroname'][end_index],neighborhoods['neighborhood'][start_index],int(1))
                yield((boroughs['boroname'][end_index].strip(),neighborhoods['neighborhood'][start_index].strip()),1)

In [ ]:
if __name__=='__main__':
    from heapq import nlargest
    rdd = sc.textFile('yellow_tripdata_2011-05.csv')\
    .mapPartitionsWithIndex(processTrips)\
    .reduceByKey(lambda x,y:x+y).take(5)
#     .map(lambda x:tuple(x[0].split(":")+[x[1]]))\
#     .groupBy(lambda x: x[0])\
#     .flatMap(lambda g: nlargest(3, g[1], key=lambda x: x[2]))\
#     .saveAsTextFile("res_opt4")

In [3]:
import rtree
import fiona.crs
import geopandas as gpd
neighborhoods = gpd.read_file('neighborhoods.geojson').to_crs(fiona.crs.from_epsg(2263))
boroughs = gpd.read_file('boroughs.geojson').to_crs(fiona.crs.from_epsg(2263))